In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')
df_raw

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\1720144281.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')


,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,125057.0,40.0,MG,NaN,QOW
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,1,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,125057.0,NaN,NaN,NaN,NaN
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,1,Unknown,NaN,NaN,NaN,D,NaN,Not Available,NaN,65003.0,NaN,NaN,NaN,NaN
3,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,1,Intravenous (not otherwise specified),6 infusions at 8 weeks interval,NaN,NaN,Y,NaN,Not Available,NaN,NaN,NaN,NaN,NaN,NaN
4,100100247,10010024,5,SS,ETANERCEPT,ETANERCEPT,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,NaN,100.0,MG,NaN,/WK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030933,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,1,Unknown,"500 mg, Daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,500.0,MG,NaN,NaN
2030934,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,1,Unknown,"40 mg, UNK",NaN,NaN,U,U,unk,NaN,NaN,40.0,MG,Injection,NaN
2030935,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,1,Unknown,"500/30 mg, daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2030936,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,1,Unknown,"80 mg, bid",NaN,NaN,Y,NaN,NaN,NaN,NaN,80.0,MG,NaN,Q12H


### Selecting only necessary columns

In [3]:
df_raw_2 = df_raw[['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai']]
df_raw_2.isna().sum()

primaryid        0
caseid           0
drug_seq         0
role_cod         0
drugname         7
prod_ai      30030
dtype: int64

### Remove drugnames with null values

In [4]:
df_raw_2 = df_raw_2[df_raw_2['drugname'].isna() == False]
df_raw_2.isna().sum()

primaryid        0
caseid           0
drug_seq         0
role_cod         0
drugname         0
prod_ai      30030
dtype: int64

## Making the Lookup Table

In [5]:
#let's make the lookup table
lookup_1 = df_raw_2[['drugname', 'prod_ai']]
lookup_1.head()

,drugname,prod_ai
0,ADALIMUMAB,ADALIMUMAB
1,ADALIMUMAB,ADALIMUMAB
2,CYCLOSPORINE,CYCLOSPORINE
3,REMICADE,INFLIXIMAB
4,ETANERCEPT,ETANERCEPT


In [6]:
len(lookup_1)

2030931

In [7]:
#Removing duplicates
lookup_1.drop_duplicates(inplace= True)
lookup_1

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\2627161921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_1.drop_duplicates(inplace= True)


,drugname,prod_ai
0,ADALIMUMAB,ADALIMUMAB
2,CYCLOSPORINE,CYCLOSPORINE
3,REMICADE,INFLIXIMAB
4,ETANERCEPT,ETANERCEPT
5,METHOTREXATE,METHOTREXATE
...,...,...
2030682,APO DILTIAZ CD,NaN
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B
2030859,MYOFLEXIN,NaN


In [8]:
#Getting the count of each drugname that gets repeated
lookup_counts = lookup_1.groupby('drugname').size().reset_index(name='Count').sort_values(by=['Count'], ascending=False)
lookup_counts = lookup_counts[lookup_counts['Count'] > 1]
lookup_counts['Count'].value_counts()

Count
2    360
3      6
Name: count, dtype: int64

In [9]:
temp_1 = list(lookup_counts['drugname'])
len(temp_1)

366

In [10]:
def removeNAs(df, temp_1):
    # Create a boolean mask using & (bitwise AND) instead of 'and'
    mask = (df['drugname'].isin(temp_1)) & (df['prod_ai'].isna())
    
    # Apply the mask to create a new column with 'Remove' or 'Keep'
    df['Action'] = df.apply(lambda row: 'Remove' if mask[row.name] else 'Keep', axis=1)
    
    return df

In [11]:
lookup_2 = removeNAs(lookup_1, temp_1)
lookup_2['Action'].value_counts()

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\1171791422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Action'] = df.apply(lambda row: 'Remove' if mask[row.name] else 'Keep', axis=1)


Action
Keep      27200
Remove      347
Name: count, dtype: int64

In [12]:
lookup_2[lookup_2['Action'] == 'Remove']

,drugname,prod_ai,Action
650182,VITAMIN D3,NaN,Remove
751297,LEVOTHYROXINE SODIUM,NaN,Remove
788032,FOLIC ACID,NaN,Remove
789602,AMBRISENTAN,NaN,Remove
792024,ATENOLOL,NaN,Remove
...,...,...,...
1664444,OLMESARTAN MEDOXOMIL,NaN,Remove
1664447,PROMETHEGAN,NaN,Remove
1664527,DOVATO,NaN,Remove
1664528,POMALYST,NaN,Remove


In [14]:
lookup_2[lookup_2['drugname'] == 'DOVATO']

,drugname,prod_ai,Action
38181,DOVATO,DOLUTEGRAVIR SODIUM\LAMIVUDINE,Keep
1664527,DOVATO,NaN,Remove


In [15]:
#Removing those values from the lookup 
lookup_2 = lookup_2[lookup_2['Action'] == 'Keep']
lookup_2

,drugname,prod_ai,Action
0,ADALIMUMAB,ADALIMUMAB,Keep
2,CYCLOSPORINE,CYCLOSPORINE,Keep
3,REMICADE,INFLIXIMAB,Keep
4,ETANERCEPT,ETANERCEPT,Keep
5,METHOTREXATE,METHOTREXATE,Keep
...,...,...,...
2030682,APO DILTIAZ CD,NaN,Keep
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE,Keep
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B,Keep
2030859,MYOFLEXIN,NaN,Keep


In [17]:
#Now replace the ones with more than 1 prod_ai with the lengthiest one
temp_2 = lookup_2.groupby('drugname').size().reset_index(name='Count').sort_values(by=['Count'], ascending= False)
temp_2

,drugname,Count
7728,"DEXTROAMPHETAMINE SACCHARATE, AMPHETAMINE ASPA...",2
3672,BISOPROLOL,2
8183,DOCONEXENT\MINERALS\VITAMINS,2
8340,DOXAZOSIN,2
13229,JEMPERLI,2
...,...,...
18,HYDROXYCHOLORQUINE,1
19,HYDROXYZINE 25 mg,1
20,Ipratroprium Bromide,1
21,Itrapropium,1


In [18]:
temp_2 = temp_2[temp_2['Count'] > 1]
temp_2

,drugname,Count
7728,"DEXTROAMPHETAMINE SACCHARATE, AMPHETAMINE ASPA...",2
3672,BISOPROLOL,2
8183,DOCONEXENT\MINERALS\VITAMINS,2
8340,DOXAZOSIN,2
13229,JEMPERLI,2
7753,DEXTROSE,2
10694,FOSAPREPITANT DIMEGLUMINE,2
816,AIMOVIG,2
242,ABIRATERONE,2
12505,ICLUSIG,2


In [19]:
#Making something similar to a window function
lookup_2['wc_len'] = lookup_2['prod_ai'].str.len()
lookup_2

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\3372529132.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_2['wc_len'] = lookup_2['prod_ai'].str.len()


,drugname,prod_ai,Action,wc_len
0,ADALIMUMAB,ADALIMUMAB,Keep,10.0
2,CYCLOSPORINE,CYCLOSPORINE,Keep,12.0
3,REMICADE,INFLIXIMAB,Keep,10.0
4,ETANERCEPT,ETANERCEPT,Keep,10.0
5,METHOTREXATE,METHOTREXATE,Keep,12.0
...,...,...,...,...
2030682,APO DILTIAZ CD,NaN,Keep,NaN
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE,Keep,34.0
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B,Keep,21.0
2030859,MYOFLEXIN,NaN,Keep,NaN


In [21]:
lookup_2['wc_rnk'] = lookup_2.groupby('drugname')['wc_len'].rank(method='dense', ascending=False)
lookup_2

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\1713614845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_2['wc_rnk'] = lookup_2.groupby('drugname')['wc_len'].rank(method='dense', ascending=False)


,drugname,prod_ai,Action,wc_len,wc_rnk
0,ADALIMUMAB,ADALIMUMAB,Keep,10.0,1.0
2,CYCLOSPORINE,CYCLOSPORINE,Keep,12.0,1.0
3,REMICADE,INFLIXIMAB,Keep,10.0,1.0
4,ETANERCEPT,ETANERCEPT,Keep,10.0,1.0
5,METHOTREXATE,METHOTREXATE,Keep,12.0,1.0
...,...,...,...,...,...
2030682,APO DILTIAZ CD,NaN,Keep,NaN,NaN
2030701,ACETYLDIHYDROCODEINE HYDROCHLORIDE,ACETYLDIHYDROCODEINE HYDROCHLORIDE,Keep,34.0,1.0
2030777,VIRAFERONPEG,PEGINTERFERON ALFA-2B,Keep,21.0,1.0
2030859,MYOFLEXIN,NaN,Keep,NaN,NaN


In [22]:
lookup_2['wc_rnk'].value_counts()

wc_rnk
1.0    10795
2.0       25
Name: count, dtype: int64

In [23]:
lookup_3 = lookup_2[lookup_2['wc_rnk'] != 2]
lookup_3['wc_rnk'].value_counts()

wc_rnk
1.0    10795
Name: count, dtype: int64

In [24]:
len(lookup_3)

27175

In [25]:
#Making NaN values in lookup NA
def fillMissingLookup(df):
    if pd.isna(df['prod_ai']):
        return 'NA'
    else:
        return df['prod_ai']

In [26]:
lookup_3['corr_prod_ai'] = lookup_3.apply(fillMissingLookup, axis=1)
lookup_3.isna().sum()

C:\Users\USER\AppData\Local\Temp\ipykernel_6884\2734754321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookup_3['corr_prod_ai'] = lookup_3.apply(fillMissingLookup, axis=1)


drugname            0
prod_ai         16380
Action              0
wc_len          16380
wc_rnk          16380
corr_prod_ai        0
dtype: int64

### Selecing the final version of the Lookup Table

In [27]:
lookup_4 = lookup_3[['drugname', 'corr_prod_ai']]
lookup_4.isna().sum()

drugname        0
corr_prod_ai    0
dtype: int64

### Filling missing prod_ai values

In [28]:
#Merging
df_raw_3 = pd.merge(left = df_raw_2, right = lookup_4, how = 'left', left_on = 'drugname', right_on='drugname')
df_raw_3

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,corr_prod_ai
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,ADALIMUMAB
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,ADALIMUMAB
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,CYCLOSPORINE
3,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,INFLIXIMAB
4,100100247,10010024,5,SS,ETANERCEPT,ETANERCEPT,ETANERCEPT
...,...,...,...,...,...,...,...
2030926,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,DAPTOMYCIN
2030927,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,PANTOPRAZOLE SODIUM
2030928,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE
2030929,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,METHYLPREDNISOLONE


In [29]:
df_raw_3.isna().sum()

primaryid           0
caseid              0
drug_seq            0
role_cod            0
drugname            0
prod_ai         30030
corr_prod_ai        0
dtype: int64